## Relational Embedding

In [68]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [69]:
from relational_embedder.data_prep import data_prep_utils as dpu
import numpy as np
from relational_embedder import api
from relational_embedder.api import Fabric
from relational_embedder.api import SIMF
import word2vec
import pickle
import itertools
path_to_row_we_model = '/data/raulcf/relemb/mitdwh/mitdwh_model/row_we_db2vec_75_50_n20_i40.bin'
row_we_model = word2vec.load(path_to_row_we_model)
path_to_col_we_model = '/data/raulcf/relemb/mitdwh/mitdwh_model/col_we_w2v_75_50_i40.bin'
col_we_model = word2vec.load(path_to_col_we_model)
path_to_row_relemb = '/data/raulcf/relemb/mitdwh/mitdwh_model/relemb/row.pkl'
with open(path_to_row_relemb, "rb") as f:
    row_relemb = pickle.load(f)
path_to_col_relemb = '/data/raulcf/relemb/mitdwh/mitdwh_model/relemb/col.pkl'
with open(path_to_col_relemb, "rb") as f:
    col_relemb = pickle.load(f)
path_to_row_hubness = '/data/raulcf/relemb/mitdwh/mitdwh_model/relemb/row_hubness.pkl'
with open(path_to_row_hubness, "rb") as f:
    word_row_hubness = pickle.load(f)
path_to_col_hubness = '/data/raulcf/relemb/mitdwh/mitdwh_model/relemb/col_hubness.pkl'
with open(path_to_col_hubness, "rb") as f:
    word_col_hubness = pickle.load(f)
fabric = Fabric(row_we_model, col_we_model, row_relemb, col_relemb, None, word_hubness=word_row_hubness)

# Find related and similar content

### testing summarizer

In [101]:
def combine(vecs_to_combine):
    #print(vecs_to_combine)
    vecs_to_combine = np.asarray(vecs_to_combine)
    comb = np.mean(vecs_to_combine, axis=0)
    return comb

In [105]:
def select_diverse_sample(vectors, k=5):
    assert len(vectors) > k

    seed_index = 0
    seed = vectors[seed_index]
    k_result = []
    k_result.append(seed_index)
    k -= 1
    while k > 0:
        sims = np.dot(vectors, seed.T)
        indexes_sorted_by_distance = np.argsort(sims)[::-1]
        most_dissimilar_index = indexes_sorted_by_distance[-1]
        most_dissimilar_metric = sims[most_dissimilar_index]
        k_result.append(most_dissimilar_index)
        k -= 1
        seed = combine([vectors[most_dissimilar_index], seed])  # we keep seed always a vector
    return k_result

In [116]:
vecs = list(fabric.RE_R['table_2.csv']['rows'].values())

In [117]:
summary = select_diverse_sample(vecs, k=5)

In [118]:
summary

[0, 1080, 1218, 1043, 12771]

In [119]:
import pandas as pd

In [120]:
df = pd.read_csv("/data/raulcf/relemb/mitdwh/bench_mitdwh_dataset/tables/table_2.csv", encoding='latin1')

In [121]:
for idx in summary:
    print(df.iloc[idx])

Unnamed: 0                       0
Mit Id                   924892098
User Name                   afahey
Full Name          Fahey, Alison C
Office Phone            6173246363
Directory Title     Policy Manager
Name: 0, dtype: object
Unnamed: 0                        1080
Mit Id                       920176588
User Name                     bperreau
Full Name               Perreau, Bruno
Office Phone                6177155384
Directory Title    Associate Professor
Name: 1080, dtype: object
Unnamed: 0                                1218
Mit Id                               900044399
User Name                               aviana
Full Name                   Viana, Alexandre N
Office Phone                        6172521809
Directory Title    Assistant Animal Technician
Name: 1218, dtype: object
Unnamed: 0                       1043
Mit Id                      900027757
User Name                      boning
Full Name             Boning, Duane S
Office Phone               6172530931
Directory

In [36]:
from difflib import SequenceMatcher
SequenceMatcher(None, 'Samuel Madden', 'Samuel R Madden').ratio()

0.9285714285714286

In [37]:
topk = api.topk_similar_vectors('Madden Samuel', k=20)
topk

[('987515930', 0.9704794477157795),
 ('database_systems', 0.7108833455379104),
 ('6_814', 0.6885508485415357),
 ('6_830', 0.6582210523292098),
 ('10000577', 0.645577892055571),
 ('norwegianwood', 0.642001062478319),
 ('fernandez_hernandez', 0.6298207117547909),
 ('maria_ledia', 0.6288562580773556),
 ('11b30964b6413d46e0533d2f0912d418', 0.6287695227795906),
 ('21w_675-students', 0.6280414608289928),
 ('wang_lie', 0.6231579323745947),
 ('924899695', 0.6230523386149919),
 ('7_ur2015su', 0.6215173178687583),
 ('rui_manuel;_sousa_ronald_(translator)', 0.6209315456905655),
 ('ox', 0.6182968548040411),
 ('44880', 0.6182031174612854),
 ('yes_comrade', 0.6181331899984215),
 ('xfb27-58', 0.617134942769223),
 ('207_08', 0.6157527175682325),
 ('900044203', 0.6148640836418144)]

In [19]:
topk = api.topk_similar_vectors('Madden Samuel', k=20, simf=SIMF.EUCLIDEAN)
topk

[('database_systems', 0.24233881571862992),
 ('987515930', 0.15369338857426773),
 ('wu_bae-ian', 0.07573126679797126),
 ('madden_sam', 0.0518519446297222),
 ('6_830-madden2012sp:6_830', 0.04127597475975742),
 ('horn_berthold', 0.03152631937939554),
 ('6_830-fa142015fa:6_830', 0.03106373849064148),
 ('triantafyllou_michael', 0.008995808753958112),
 ('6_830-sp132013sp:6_830', 0.0017528920682097215),
 ('hoyt_j', -0.005339910261132452),
 ('hagelstein_peter', -0.009514236230763418),
 ('leeb_steven', -0.01571654072645856),
 ('6_814-sp132013sp:6_814', -0.019446027407990618),
 ('6_814-fa142015fa:6_814', -0.021701738894042588),
 ('karger_david', -0.031132061019172763),
 ('orlando_terry', -0.03163186466972512),
 ('aaronson_scott', -0.03603093948061531),
 ('6_830-madden2010fa:6_830', -0.03745892374626969),
 ('6_840-fa142015fa:6_840', -0.038542739368728096),
 ('ram_rajeev', -0.03879925252548766)]

In [38]:
sim = api.similarity_between('Madden Samuel', 'Madden Samuel R')
sim

-0.1845713624392913

In [39]:
sim = api.similarity_between('Madden Samuel', 'Madden Samuel R', simf=SIMF.EUCLIDEAN)
sim

-0.5392019766354845

In [40]:
# observe what happens when i introduce the id
res = api.topk_similar_vectors('987515930', k=10)
res

[('madden_samuel', 0.9704794477157795),
 ('database_systems', 0.6813900021727747),
 ('6_814', 0.6522947536607404),
 ('6_830', 0.6375059438707475),
 ('900044203', 0.6362122372250121),
 ('nocera_daniel', 0.6240305364077539),
 ('wang_lie', 0.611838581644423),
 ('10000577', 0.6113988263143069),
 ('rui_manuel;_sousa_ronald_(translator)', 0.6026981696788847),
 ('11b30964b6413d46e0533d2f0912d418', 0.59862216885253)]

In [57]:
vec = api.vector_for_entity('Madden Samuel R')
topk = api.topk_relations(vec, k=25)
topk

[('Sis_course_description.csv', 0.13821265493723256),
 ('Hr_faculty_roster.csv', 0.06763860381512055),
 ('short_course_catalog_subject_offered.csv', -0.006685810130188097),
 ('Person_auth_area.csv', -0.014319455379207646),
 ('short_cis_course_catalog.csv', -0.014579377421317208),
 ('Drupal_employee_directory.csv', -0.0553644114491505),
 ('Moira_list.csv', -0.05790861336472286),
 ('short_subjects_offered.csv', -0.06257802428233639),
 ('Iap_subject_category.csv', -0.07795257067822159),
 ('Se_person.csv', -0.07847075535440196),
 ('Sdo_datum_vertical.csv', -0.08425240822342683),
 ('Sdo_datums_old_format.csv', -0.08829224836030192),
 ('Fac_rooms.csv', -0.09103996550109894),
 ('Student_degree_program.csv', -0.0911096304126977),
 ('Dr_object_attribute.csv', -0.09220820847626654),
 ('Warehouse_users.csv', -0.09628984392411777),
 ('Fac_building_address.csv', -0.09661750331865804),
 ('Fclt_rooms.csv', -0.0974196574751569),
 ('Sdo_datum_engineering.csv', -0.09813562536399223),
 ('short_subject_su

In [42]:
vec = api.vector_for_entity('Madden Samuel')
topk = api.topk_relations(vec, k=5, simf=SIMF.EUCLIDEAN)
topk

[('Num.csv', 0.16936896528114032),
 ('Star_user_view.csv', 0.1655799439870267),
 ('Space_usage.csv', 0.14747305262391708),
 ('Student_department.csv', 0.14498072458483835),
 ('Cip_with_version.csv', 0.14222394923954174)]

In [53]:
vec = api.vector_for_entity('Madden Samuel')
topk = api.topk_columns(vec, k=6)
topk

[('Mit Id', 'Hr_faculty_roster.csv', 0.639144706836574),
 ('Instructor Name', 'Library_course_instructor.csv', 0.6311655512861237),
 ('Responsible Faculty Name',
  'Library_subject_offered.csv',
  0.6294495824797028),
 ('Responsible Faculty Mit Id',
  'Library_subject_offered.csv',
  0.6287341826679927),
 ('Mit Id', 'Space_supervisor_usage.csv', 0.5971622603744144),
 ('Responsible Faculty Mit Id',
  'short_tip_subject_offered.csv',
  0.5608406749654329)]

In [8]:
vec = api.vector_for_entity('Madden Samuel')
topk = api.topk_columns(vec, k=6, simf=SIMF.EUCLIDEAN)
topk

[('Responsible Faculty Name',
  'Library_subject_offered.csv',
  0.14355357542106928),
 ('Instructor Name', 'Library_course_instructor.csv', 0.13059531855122375),
 ('Responsible Faculty Name',
  'short_tip_subject_offered.csv',
  0.08328505294699973),
 ('Library Subject Offered Key',
  'Library_subject_offered.csv',
  0.03323005816263902),
 ('Library Course Instructor Key',
  'Library_course_instructor.csv',
  0.029184951346477694),
 ('Shift X', 'Sdo_datums_old_snapshot.csv', 0.022234865679014004)]

In [ ]:
vec = api.vector_for_entity('Madden Samuel')
topk = api.topk_rows(vec, k=5)
topk

In [43]:
res = api.analogy('madden_samuel_r', '32-g936', '32-g938')
res

[('ne49-4122a-201601', 0.20538742589397418),
 ('nw86-812-201603', 0.1982128485954503),
 ('e40-196l-201601', 0.19270325473588984),
 ('1-235l-201606', 0.18637941400349384),
 ('32-g918-201509', 0.1862798032592471),
 ('w98-056a-201506', 0.18382321257095294),
 ('e19-041-201511', 0.18090040199523624),
 ('nw22-257-201603', 0.17879917726476208),
 ('w85-1510-201506', 0.17861705166973985),
 ('w85-1109-201506', 0.17790929505750713)]

In [44]:
topk = api.topk_similar_vectors('madden_samuel_r', k=10)
topk

[('madden@csail_mit_edu', 0.9877514181424898),
 ('6172586643', 0.9661914500477062),
 ('http://db_csail_mit_edu/madden', 0.9601653803020539),
 ('joe_edwards@ll_mit_edu', 0.9155993249517854),
 ('7819815402', 0.9050343751460893),
 ('922483482', 0.9024991889036602),
 ('ll-fr2-523', 0.902453853359129),
 ('7819814855', 0.9017216264757932),
 ('928294153', 0.9017030609242505),
 ('wulf', 0.900780053532261)]

In [58]:
res = api.concept_qa('madden_samuel_r', 'Employee_directory.csv', 'Email Address', n=10)
res

[('6172586643', -0.12508617457917892),
 ('http://db_csail_mit_edu/madden', -0.1391751966904704),
 ('madden@csail_mit_edu', -0.17201634098444663),
 ('ll-fr2-523', -0.17684236222341787),
 ('7819815402', -0.1777956857099776),
 ('928294153', -0.18409857307346034),
 ('joe_edwards@ll_mit_edu', -0.22403632343823232),
 ('922483482', -0.2797531086329541),
 ('7819814855', -0.28201589241208014),
 ('wulf', -0.3011764034743578)]

In [ ]:
res = api.concept_qa('madden_samuel_r', 'Employee_directory.csv', 'Email Address', n=10, simf=SIMF.EUCLIDEAN)
res

In [62]:
res = api.concept_qa_denoising('madden_samuel_r', 'Employee_directory.csv', 'Email Address', denoise_heuristic=20, n=30)
res

[]

In [ ]:
res = api.concept_qa('968548423', 'Employee_directory.csv', 'Office Phone', n=10)
res

In [ ]:
res = api.concept_qa_denoising('968548423', 'Employee_directory.csv', 'Office Phone', n=10)
res

In [ ]:
res = api.concept_qa('kimball@mit.edu', 'Employee_directory.csv', 'Office Phone', n=5)
res

In [ ]:
res = api.concept_qa_denoising('kimball@mit.edu', 'Employee_directory.csv', 'Office Phone', n=5)
res

### Exploring row-vec relation

**Mit Id, Last Name, First Name, Middle Name, Full Name, Office Location, Office Phone, Department Number, Department Name, Krb Name, Email Address, Personal Url**

*968548423, Kimball, Richard, W, Kimball,Richard W, 3-269, 6172539707, 65000 Dep. of Mechanical Engineering, kimball, kimball@mit.edu, (no url)*

In [50]:
res = api.topk_similar_vectors('968548423', k=10)
res

[('kimball_richard', 0.8274650400385563),
 ('alexander', 0.5808724022246857),
 ('johnson_nathan_d', 0.5766780708015954),
 ('professor_of_the_practice', 0.5752395511249825),
 ('scott_w', 0.5746770036744993),
 ('mmimee', 0.5742350804025869),
 ('johnson_nathan', 0.5721213664985816),
 ('donalpjc', 0.5715372122925946),
 ('jpryor', 0.5711173065619926),
 ('nathanj', 0.5701231829015815)]

In [51]:
res = api.topk_similar_vectors('kimball_richard_w', k=10)
res

[('kimball@mit_edu', 0.9957214668345413),
 ('6172539707', 0.9953959078943915),
 ('kimball', 0.9744170385852329),
 ('wang_richard_y', 0.9669832858426947),
 ('richard_y', 0.966906674368178),
 ('ben', 0.9632443268108364),
 ('ellison_sara_f', 0.9614352546871556),
 ('sara_f', 0.9613979800455246),
 ('atissa', 0.9608966758425126),
 ('banuazizi_atissa', 0.9601458521929616)]

In [52]:
res = api.topk_similar_vectors('6172539707', k=10)
res

[('kimball@mit_edu', 0.9965228152447645),
 ('kimball_richard_w', 0.9953959078943915),
 ('kimball', 0.97059189457399),
 ('wang_richard_y', 0.9647554160654629),
 ('richard_y', 0.964629295976174),
 ('ben', 0.9609904078856978),
 ('sara_f', 0.9588804402276588),
 ('ellison_sara_f', 0.9587795636030549),
 ('440102_0', 0.9585031610227653),
 ('sea_grant_college_program', 0.9561181442812635)]

In [12]:
res = api.topk_similar_vectors('kimball', k=10)
res

[('968548423', 0.5483184600358931),
 ('kimball_richard_w', 0.5068739904779622),
 ('kimball_township', 0.4986258990399113),
 ('hawaii-noj', 0.4944372859818194),
 ('orc_sm', 0.49344520751537957),
 ('hicklin2', 0.468650574692114),
 ('55353', 0.4678833197210878),
 ('910006165', 0.46323332227903713),
 ('acl-ror-coalition', 0.457991460737341),
 ('7819811585', 0.45526744093539717)]

In [14]:
res = api.topk_similar_vectors('3-269', k=10)
res

[('3-2-269', 0.6446704024315779),
 ('6172539707', 0.5979994035741272),
 ('kimball_richard_w', 0.5736798852265613),
 ('3-2-267e', 0.5620825672752245),
 ('784_24', 0.5603887666061109),
 ('3-269-201505', 0.5138520039849235),
 ('3-269a', 0.5024417504873565),
 ('968548423', 0.4931682849509945),
 ('3-267e', 0.4921089645404333),
 ('5-223e', 0.46351824567559924)]

In [15]:
res = api.topk_similar_vectors('richard', k=10)
res

[('david', 0.6014876246375217),
 ('john', 0.5896711580842966),
 ('robert', 0.5804079490265617),
 ('michael', 0.570901838852631),
 ('william', 0.5456058126832005),
 ('christopher', 0.542281778567466),
 ('stephen', 0.5417686252204659),
 ('george', 0.5407281947881049),
 ('james', 0.5370358067535486),
 ('daniel', 0.535614185293984)]

In [16]:
res = api.topk_similar_vectors('w', k=10)
res

[('m', 0.5063437823276751),
 ('r', 0.5029316132638273),
 ('j', 0.4815215970814507),
 ('a', 0.46914693881402625),
 ('f', 0.46682928486374387),
 ('david', 0.45239585840451824),
 ('richard', 0.4523346174822213),
 ('01/08/16_01:16_am', 0.44950321014075567),
 ('e', 0.4424572230642748),
 ('benjamin', 0.43901658968098345)]

In [ ]:
topk = api.topk_columns(api.vector_for_entity('6172539707'), k=5)
topk

In [ ]:
topk = api.topk_columns(api.vector_for_entity('kimball@mit.edu'), k=5)
topk

In [ ]:
res = api.concept_expansion('kimball@mit.edu', 'Employee_directory.csv', 'Email Address', k=5)
res

In [ ]:
import word2vec
import itertools
import numpy as np
from relational_embedder import composition
from data_prep import data_prep_utils as dpu
from scipy.spatial.distance import cosine
import pandas as pd

In [ ]:
# Loading binary vectors
model = word2vec.load("/home/ubuntu/word2vec_c/data/mitdwhdata.bin")

In [ ]:
indexes, metrics = model.cosine('madden_samuel_r', n=20)
res = model.generate_response(indexes, metrics).tolist()
res

In [ ]:
indexes, metrics = model.cosine('32-g938', n=20)
res = model.generate_response(indexes, metrics).tolist()
res

#### Basic Analogies

In [ ]:
indexes, metrics = model.analogy(pos=['madden_samuel_r', '32-g936'], neg=['32-g938'], n=10)

In [ ]:
res = model.generate_response(indexes, metrics).tolist()
res

#### Analogies across Relations

In [ ]:
indexes, metrics = model.analogy(pos=['gallop_sarah_e', '10-219'], neg=['11-245'], n=10)
res = model.generate_response(indexes, metrics).tolist()
res

## Baseline composition: Columns and Relations

In [ ]:
# Files
import os
path = "/data/datasets/mitdwh/"
all_relations = [relation for relation in os.listdir(path)]

In [ ]:
composition_vectors = dict()

In [ ]:
for relation in all_relations:
    print("Computing vectors for: " + str(relation))
    col_we, missing_words = composition.column_avg_composition(path + "/" + relation, model)
    rel_we = composition.relation_column_composition(col_we)
    composition_vectors[relation] = rel_we
    for k, v in col_we.items():
        composition_vectors[relation +"." + k] = col_we[k]
print("Done!")
print("Total vectors: " + str(len(composition_vectors.items())))

In [ ]:
# SERIALIZE
import pickle
path = "./temp/composition_vectors.pkl"
with open(path, 'wb') as f:
    pickle.dump(composition_vectors, f)

In [ ]:
# DESERIALIZE
import pickle
path = "./temp/composition_vectors.pkl"
with open(path, 'rb') as f:
    composition_vectors = pickle.load(f)

In [ ]:
def similar_to(table, column=None, threshold=0.2):
    similar_items = []
    if column is None:
        key = table
    else:
        key = table + "." + column
    for k, we in composition_vectors.items():
        distance = 0
        try:
            distance = cosine(composition_vectors[key], composition_vectors[k])
        except ValueError:
            continue
        if distance < threshold:
            similar_items.append(k)
    return similar_items

In [ ]:
table = "Fac_building.csv"
column = "Building Name Long"
#key = "Tables.csv.Business Contact Email"
threshold = 0.2
sim_items = similar_to(table, column=column)
for el in sim_items:
    print(str(el))

In [ ]:
table = "Library_subject_offered.csv"
column = "Course Number Desc"
sim_items = similar_to(table, column)
for el in sim_items:
    print(str(el))

In [ ]:
table = "Warehouse_users.csv"
sim_items = similar_to(table)
for el in sim_items:
    print(str(el))

In [ ]:
table = "Fac_building.csv"
sim_items = similar_to(table, threshold=0.1)
for el in sim_items:
    print(str(el))

### Other examples

In [ ]:
path_to_relation = "/data/datasets/mitdwh/Se_person.csv"
col_we_se, missing_words = composition.column_avg_composition(path_to_relation, model)

In [ ]:
path_to_relation = "/data/datasets/mitdwh/Drupal_employee_directory.csv"
col_we_drupal, missing_words = composition.column_avg_composition(path_to_relation, model)

In [ ]:
se_vec = composition.relation_column_composition(col_we_se)
drupal_vec = composition.relation_column_composition(col_we_drupal)

In [ ]:
cosine(se_vec, drupal_vec)

In [ ]:
for a, b in itertools.combinations(col_we_se.keys(), 2):
    we_a = col_we_se[a]
    we_b = col_we_se[b]

    cos = cosine(we_a, we_b)
    print(str(a) + " -sim- " + str(b) + " is: " + str(cos))

## Composition Rows

In [ ]:
path_to_relation = "/data/datasets/mitdwh/Se_person.csv"
row_we, m_words = composition.row_avg_composition(path_to_relation, model)

In [ ]:
df = pd.read_csv(path_to_relation, encoding='latin1')

In [ ]:
row_ref = 0
distances = []
for i in range(len(row_we)):
    distance = cosine(row_we[row_ref], row_we[i])
    distances.append((i, distance))
distances = sorted(distances, key=lambda x: x[1])

In [ ]:
len(distances)

In [ ]:
k = 10
topk = []
for i in range(k):
    idx = distances[i]
    row = df.iloc[idx[0]]
    topk.append((row, distances[i]))

In [ ]:
df.iloc[0]

In [ ]:
topk[1]

In [ ]:
topk[2]

In [ ]:
topk[3]

In [ ]:
topk[4]

## Operations across hierarchy

In [ ]:
def topk_similar(vec, composition_vectors):
    distances = []
    errors = 0
    for key, value in composition_vectors.items():
        try:
            distance = cosine(vec, value)
        except ValueError:
            errors += 1
            continue
        distances.append((key, distance))
    distances = sorted(distances, key=lambda x: x[1])
    return distances, errors

In [ ]:
entity = 'madden_samuel_r'
vec = model.get_vector(entity)

In [ ]:
distances, errors = topk_similar(vec, composition_vectors)
print("ERRORS: " + str(errors))

In [ ]:
for i in range(10):
    print(str(distances[i]))

#### Check rows

In [ ]:
distances = []
for i in range(len(row_we)):
    distance = cosine(vec, row_we[i])
    distances.append((i, distance))
distances = sorted(distances, key=lambda x: x[1])

In [ ]:
k = 10
topk = []
for i in range(k):
    idx = distances[i]
    row = df.iloc[idx[0]]
    topk.append((row, distances[i]))

In [ ]:
topk[0]

In [ ]:
topk[1]

In [ ]:
topk[2]

In [ ]:
topk[3]

### Designing API

In [ ]:
def concept_qa(entity, attribute, n=20):
    #vec_entity = model.get_vector(entity)
    indexes, metrics = model.cosine(entity, n=n)
    res = model.generate_response(indexes, metrics).tolist()
    vec_attribute = composition_vectors[attribute]
    candidate_attribute_sim = []
    for e, score in res:
        vec_e = model.get_vector(e)
        distance = cosine(vec_e, vec_attribute)
        candidate_attribute_sim.append((e, distance))
    candidate_attribute_sim = sorted(candidate_attribute_sim, key=lambda x: x[1])
    return candidate_attribute_sim

def entity_to_attribute(entity, n=20):
    indexes, metrics = model.cosine(entity, n=n)
    res = model.generate_response(indexes, metrics).tolist()
    res_attr = []
    for e, score in res:
        vec_e = model.get_vector(e)
        distances, errors = topk_similar(vec_e, composition_vectors)
        res_attr.append((e, score, distances[:4]))
    return res_attr

In [ ]:
# finding attributes
attribute = 'phone'
candidates = []
for key in composition_vectors.keys():
    if (key.lower()).find(attribute) != -1:
        candidates.append(key)
candidates

In [ ]:
indexes, metrics = model.cosine('madden_samuel_r', n=20)
res = model.generate_response(indexes, metrics).tolist()
res

In [ ]:
entity_to_attribute('madden_samuel_r', n=20)

In [ ]:
concept_qa('madden_samuel_r', 'Employee_directory.csv.Krb Name', n=10)

# Testing

In [18]:
import numpy as np
from scipy.spatial.distance import cosine

In [4]:
topk = api.topk_similar_vectors('Madden Samuel R', k=20)
topk

[('madden@csail_mit_edu', 0.9877514181424898),
 ('6172586643', 0.9661914500477062),
 ('http://db_csail_mit_edu/madden', 0.9601653803020539),
 ('joe_edwards@ll_mit_edu', 0.9155993249517854),
 ('7819815402', 0.9050343751460893),
 ('922483482', 0.9024991889036602),
 ('ll-fr2-523', 0.902453853359129),
 ('7819814855', 0.9017216264757932),
 ('928294153', 0.9017030609242505),
 ('wulf', 0.900780053532261),
 ('russmc@haystack_mit_edu', 0.8999511868812413),
 ('sullivan_cynthia_r', 0.899213914684368),
 ('919236452', 0.8988618799131842),
 ('marc_gauthier@ll_mit_edu', 0.8987411198249429),
 ('929832456', 0.8981311620209751),
 ('mcwhirter', 0.8977569603224066),
 ('970671073', 0.8934989928928403),
 ('7819816292', 0.8928531658222272),
 ('cindys@ll_mit_edu', 0.8927173309249392),
 ('wulf_karen_r', 0.8915586661966363)]

In [26]:
e1 = api.M.get_vector(dpu.encode_cell('Madden Samuel R'))
e2 = api.M.get_vector('madden@csail_mit_edu')
e3 = api.M.get_vector('6172586643')

In [27]:
mean_v = np.mean([e1, e3], axis=0)

In [28]:
d1 = cosine(e1, mean_v)
d1

0.008488161934587568

In [29]:
d2 = cosine(e3, mean_v)
d2

0.008488161934587679

In [33]:
mean_v2 = np.mean([e1, e2], axis=0)

In [34]:
cosine(e1, mean_v2)

0.003066848243451603

In [35]:
cosine(e2, mean_v2)

0.003066848243451714